#### Loading packages

In [1]:
import pandas as pd
import numpy as np
import itertools
from pulp import *

import warnings
warnings.filterwarnings("ignore")

#### Load data

In [2]:
df = pd.read_excel("Heat Exchanger Assignment Optimization.xlsx", sheet_name="Installation Costs")

df.set_index("Streams", inplace=True)

df

,Exchanger_1,Exchanger_2,Exchanger_3,Exchanger_4,Exchanger_5,Exchanger_6,Exchanger_7,Exchanger_8,Exchanger_9,Exchanger_10
Streams,,,,,,,,,,
Stream_1,949,1176,1073,1042,1137,999,993,944,1052,855
Stream_2,1127,1108,1163,1032,1181,1168,989,1180,841,889
Stream_3,1088,936,966,943,1034,1118,1091,1156,829,1110
Stream_4,920,917,1139,817,1154,1141,1150,821,826,969
Stream_5,907,832,905,834,1055,1135,986,852,857,813
Stream_6,1121,877,1026,878,861,1026,827,1053,1162,957
Stream_7,989,1102,1157,1175,963,957,941,1087,1041,1182
Stream_8,1060,1126,845,854,962,818,851,882,895,1172
Stream_9,1147,1168,1186,1183,983,1100,1101,905,1092,1051


#### Create dictionary of decision variables

In [3]:
streams = df.index.to_list()

exchanger_stations = df.columns.to_list()

var_dict = LpVariable.dicts(name="assign",
                            indexs=[(i, j) for i in streams for j in exchanger_stations], 
                            lowBound=0,
                            cat="Binary")

#### Define objective function

In [11]:
model = LpProblem("Assigning Streams to Heat Exchangers Ensuring Minimum Installation Costs", LpMinimize)

model += lpSum([df.loc[(i,j)]*var_dict[(i,j)] for i in streams for j in exchanger_stations])

#### Define constraints

In [12]:
# Each stream must be assigned to exactly 1 heat exchanger
for i in streams:
    model += lpSum([var_dict[(i, j)] for j in exchanger_stations]) == 1

# Every heat exchanger must receive input from exactly 1 stream    
for j in exchanger_stations:
    model += lpSum([var_dict[(i, j)] for i in streams]) == 1

#### Obtain optimal solution

In [13]:
model.solve()

if LpStatus[model.status]=="Optimal":
    optimal_soln = pd.DataFrame([(v.name, v.varValue) for v in model.variables() if v.varValue==1],
                                columns=["Assignment", "Status"])

#### Save results

In [7]:
writer = pd.ExcelWriter("Heat Exchanger Assignment Optimization.xlsx", engine='openpyxl')

df.reset_index().to_excel(writer, "Installation Costs", index=False)
optimal_soln.to_excel(writer, "Optimal Solution", index=False)

writer.save()

In [14]:
optimal_soln

,Assignment,Status
0,"assign_('Stream_1',_'Exchanger_1')",1.0
1,"assign_('Stream_2',_'Exchanger_7')",1.0
2,"assign_('Stream_3',_'Exchanger_3')",1.0
3,"assign_('Stream_4',_'Exchanger_4')",1.0
4,"assign_('Stream_5',_'Exchanger_2')",1.0
5,"assign_('Stream_6',_'Exchanger_5')",1.0
6,"assign_('Stream_7',_'Exchanger_6')",1.0
